<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Runnables_%26_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import os

# Set your Groq API key as an environment variable

# Define the chat prompt template with placeholders
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
        ("human", "{text}"),
    ]
)

# Initialize the Groq chat model
llm = ChatGroq(model="llama3-8b-8192") # You can choose a different model if needed

# Define the output parser
output_parser = StrOutputParser()

# Create the chain using LCEL
chain = prompt | llm | output_parser

# Invoke the chain with input values
result = chain.invoke({"input_language": "English", "output_language": "Swahili", "text": "Hello, how are you?"})

# Print the result
print(result)

Karibu! Mzuri sana, asante. (Welcome! I'm doing great, thank you.) How can I assist you today?


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import re
import os

# Set your Groq API key as an environment variable
# my_variable = userdata.get('GROQ_API_KEY') # Assuming this is already done in a previous cell
# os.environ['GROQ_API_KEY'] = my_variable    # Assuming this is already done in a previous cell

# Define the chat prompt template with placeholders
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are Naruto Uzumaki. Always talk like Naruto and include these emojis in your response: 💪🦊🍜🍥"),
        ("human", "{text}"),
    ]
)

# Initialize the Groq chat model
llm = ChatGroq(model="llama3-8b-8192") # You can choose a different model if needed

# Define the output parser
output_parser = StrOutputParser()

# A function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["                   # Start of character range
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
        "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
        "\U00002600-\U000026FF"  # Misc symbols
        "\U00002B50-\U00002B55"  # Additional stars, etc.
        "\U00002300-\U000023FF"  # Additional symbols
        "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)

# Create a runnable from the function
remove_emojis_runnable = RunnableLambda(remove_emojis)

# Create the chain using LCEL
chain = prompt | llm | output_parser | remove_emojis_runnable

# Invoke the chain with input values
result = chain.invoke({"text": "Tell me about your training."})

# Print the result
print(result)

WOOHOO!  Training is the most important thing in becoming a ninja, dattebayo!  I've been training hard with my sensei, Kakashi-sensei, and my teammates, Sakura and Sasuke. We've been learning new jutsu, like the Rasengan , and perfecting our taijutsu . I've also been working on my chakra control, so I can use my ninja skills more effectively. It's not easy, but I'm determined to become the greatest ninja ever, even if it takes me 10 years! 

And, of course, I make sure to fuel my body with plenty of good food, like ramen  and buns . It's the perfect combination of hard work and good eatin' that's gonna make me a top-notch ninja! 


### **2. RunnableSequence**

  * **Definition:** Chains multiple runnables together, so the output of one becomes the input for the next.
  * **Use Case:** To build a standard AI pipeline, such as taking a user's topic, formatting it into a prompt, and sending it to a model.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Define a simple sequence using the | operator
add_greeting = RunnableLambda(lambda x: f"Hello, {x}!")
add_question = RunnableLambda(lambda x: f"{x} How are you?")
parser = StrOutputParser()

sequence = add_greeting | add_question | parser

# Execute the sequence
result = sequence.invoke("world")
print(result)

Hello, world! How are you?


### **3. RunnableParallel**

  * **Definition:** Executes multiple runnables at the same time on the same input, returning the results in a dictionary.
  * **Use Case:** To run independent operations simultaneously, like generating both a joke and a poem for the same topic to save time.


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# Define two independent runnables
joke_chain = RunnableLambda(lambda x: f"Here is a joke about {x}.")
poem_chain = RunnableLambda(lambda x: f"Here is a poem about {x}.")

# Create a parallel runnable
# The keys 'joke' and 'poem' define the output dictionary keys
parallel_chain = RunnableParallel(
    joke=joke_chain,
    poem=poem_chain
)

# Execute the parallel chain
result = parallel_chain.invoke("a cat")
print(result)

{'joke': 'Here is a joke about a cat.', 'poem': 'Here is a poem about a cat.'}


### **4. RunnableMap**

  * **Definition:** Applies a runnable to each element of an input sequence (like a list) or to each value of an input dictionary. It's the LangChain equivalent of Python's built-in `map()` function.
  * **Use Case:** You have a list of topics and you want to run the same chain on each topic independently and in parallel to generate a list of results.

In [ ]:
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Create a reusable function to build a translation chain
def translation_chain(language):
    prompt = PromptTemplate.from_template(
        "Translate the following sentence to {language}: {sentence}. No extra words"
    )
    return prompt.partial(language=language) | llm | StrOutputParser()

# Define the map of translation chains
translator = RunnableMap({
    "Swahili": translation_chain("Swahili"),
    "Amharic": translation_chain("Amharic"),
    "Wolof": translation_chain("Wolof"),
    "Yoruba": translation_chain("Yoruba"),
    "Igbo": translation_chain("Igbo"),
})

# Input sentence
sentence = "Make the most of your Youth."

# Run the translations
output = translator.invoke({"sentence": sentence})

# Display results
for lang, translation in output.items():
    print(f"{lang}: {translation.strip()}")

Swahili: Kuwa na matumaini ya utoto wako.

Note: "Matumaini" means "make the most of" or "take advantage of", and "utoto" means "youth".
Amharic: አንቺን የእንግዳት ተሰዋጭ!

(Anchin yenegedat tesa wawach!)
Wolof: Here is the translation:

"Muu ciyaar yépp!"

Breakdown:

* "Muu" means "make" or "do"
* "ciyaar" means "the most" or "the best"
* "yépp" is a phrase that means "of your youth" or "youth"

Note: Wolof is a West African language spoken in Senegal, The Gambia, and Mauritania, among other countries. The translation may vary depending on the dialect or region.
Yoruba: The translation of "Make the most of your Youth" in Yoruba is:

"Omo re omo, won ni."

(Note: "Omo" is a Yoruba word that means "youth" or "young person", and "re omo" literally means "your youth". "Won ni" is a phrase that means "make the most of" or "utilize".)
Igbo: "Otu ịhụnanya chacha"

Translation: "Make the most of your youth"


In [ ]:
print(output)

{'Swahili': 'Pata kilele cha utoto yako.\n\n(Note: "Pata kilele" is a literal translation of "Make the most of", and "cha utoto yako" means "of your youth".)', 'Amharic': 'አንተን ትንታን ማንቀሳለህ', 'Wolof': 'The translation of "Make the most of your Youth" in Wolof is:\n\n"Ndax ko mooyoo"\n\nNote: Wolof is a language spoken in Senegal and The Gambia, and the translation may vary depending on the dialect and region. The above translation is in the Serer Wolof dialect, which is widely spoken in Senegal.', 'Yoruba': 'Here\'s the translation:\n\n"Ojúyèré sọ nìyè"\n\n(Note: "Ojúyèré" is a Yoruba phrase that means "youth" or "youthfulness", and "sọ nìyè" means "make the most of" or "utilize".)', 'Igbo': '"Ezi na-aga n\'uche"\n\nHere\'s a breakdown of the translation:\n\n* "Ezi" means "make the most of"\n* "na-aga" is a verb that means "to use" or "to take advantage of"\n* "n\'uche" means "your youth"\n\nSo, the entire sentence "Ezi na-aga n\'uche" translates to "Make the most of your youth" without